In [51]:
[all  true swap [&] step] inscribe
[any false swap [|] step] inscribe

[0 3 1]

In [52]:
[base 2147483648] inscribe

[0 3 1]

Model bigints as a pair of Boolean for the sign and a list of integers for the digits, to keep things simple let the bool be the first item on a list followed by zero or more int digits.  The digits shall be confined to the range zero to `pow(2, 31) - 1`

Another way to say that is that our numbers are in base `2147483648` and our "nine" is `2147483647` (`0b1111111111111111111111111111111`, 31 ones.)

This lets us use (Oberon RISC) 32-bit signed ints to store our digits.

## Take Two

This really got away from me the first time.  I think the big mistake ai made was including the middle branch in the recursive branch.


In [53]:
[valid_digit [0 >] [base <] &&] inscribe

[0 3 1]

In [54]:
clear
32 valid_digit 1232147483648 valid_digit

32 true 1232147483648 false

In [55]:
clear
[true 3 2 1]
rest [valid_digit] map all

true

In [56]:
clear
[true 3 2 1]

[true 3 2 1]

Because we are working with Python Joy right now we can convert ints to bigints.

To get the sign bool we can just use `!-` ("not negative")

In [57]:
clear
23 !- -23 !-

true false

To get the list of digits...

```
    @staticmethod
    def digitize(n):
        if n < 0:
            raise ValueError(f'Non-negative only: {n}')
        while n:
            n, digit = divmod(n, 2**31)
            yield OberonInt(digit)

```

    [0 >=] [base divmod swap] while

In [58]:
clear

12345678901234567890 base divmod swap

1797196498 5748904729

In [59]:
base divmod swap

1797196498 1453937433 2

In [60]:
clear

1234567890123456789012345678901234567890
   [0 >] [base divmod swap] while pop

1312754386 1501085485 57659106 105448366 58

In [61]:
clear

[1234567890123456789012345678901234567890]
   [ [0 >] [base divmod swap] while pop ]
   infra

[58 105448366 57659106 1501085485 1312754386]

In [62]:
clear

1234567890123456789012345678901234567890
   [0 <=] [pop []] [base divmod swap] [i cons]

1234567890123456789012345678901234567890 [0 <=] [pop []] [base divmod swap] [i cons]

In [63]:
genrec

[1312754386 1501085485 57659106 105448366 58]

In [64]:
clear

0 [0 <=] [pop []] [base divmod swap] [i cons] genrec

[]

In [65]:
clear

[digitalize [0 <=] [pop []] [base divmod swap] [i cons] genrec] inscribe

In [66]:
clear

1234567890123456789012345678901234567890 digitalize

[1312754386 1501085485 57659106 105448366 58]

So `!-` for the sign and `abs digitalize` for the digits, followed by `cons`:

In [67]:
clear

1234567890123456789012345678901234567890 [!-] [abs digitalize] cleave cons

[true 1312754386 1501085485 57659106 105448366 58]

In [68]:
[to_bigint [!-] [abs digitalize] cleave cons] inscribe

[true 1312754386 1501085485 57659106 105448366 58]

In [69]:
clear

1234567890123456789012345678901234567890 to_bigint

[true 1312754386 1501085485 57659106 105448366 58]

In [70]:
clear

-1234567890123456789012345678901234567890 to_bigint

[false 1312754386 1501085485 57659106 105448366 58]

In [71]:
clear

## Addition of Like Signs `add-digits`

Let's figure out how to add two lists of digits (we will assume that the signs are the same.)  We need to put an inital `false` value for a carry flag, and then there's a `genrec`.

    initial-carry ≡ false rollup
      add-digits' ≡ [P] [THEN] [R0] [R1] genrec
      
       add-digits ≡ initial-carry add-digits'

### The predicate

I think we're going to want a recursive function (duh!?) but it's not quite a standard *hylomorphism* for (at least) two reasons:

- We're tearing down two lists simultaneously.
- They might not be the same length.

There are two base cases: two empty lists or one empty list, the recursive branch is taken only if both lists at non-empty.

    bool [a ...] [b ...] P

The first thing to do is convert them to Booleans:

In [72]:
clear
[
[[a] [b]]
[[a] []]
[[] [b]]
[[] []]
]
[[[bool] ii] infra] 
map

[[true true] [true false] [false true] [false false]]

In [73]:
clear
[
[[a] [b]]
[[a] []]
[[] [b]]
[[] []]
]
[[[bool] ii & not] infra] 
map

[[false] [true] [true] [true]]

    P == [bool] ii & not

### The base cases
We have to decide between three cases, but because addition we can lump together the first two cases:

    bool [] [b ...] THEN
    bool [a ...] [] THEN
    bool [] [] THEN

    THEN ≡ [P'] [THEN'] [ELSE] ifte

In [74]:
clear
[
[[a] []]
[[] [b]]
[[] []]
]
[[[bool] ii |] infra] 
map

[[true] [true] [false]]

    P' ≡ [bool] ii |

So `THEN'` deals with one number (list of digits) being longer than the other, while the `ELSE` branch deals with the case of both lists being empty.

#### One list empty
In the cases where one of the two lists (but not both) is empty:

    carry [a ...] [] THEN'
    carry [] [b ...] THEN'

We first get rid of the empty list:

    ditch-empty-list ≡ [bool] [popd] [pop] ifte

In [75]:
clear
[ditch-empty-list [bool] [popd] [pop] ifte] inscribe

In [76]:
[1][] ditch-empty-list

[1]

In [77]:
clear
[][1] ditch-empty-list

[1]

    THEN' ≡ ditch-empty-list THEN''

Now we have:

    carry [n ...] THEN''

This is `add-carry-to-digits` that kicked my ass earlier today.  Like I mentioned above, I think it was because I put it in the recursive branch!  D'oh!

#### But first  `add-with-carry`
We will want some function `F` that accepts a bool and two ints and leaves behind a new int and a new Boolean carry flag:

            carry0 a b F
    --------------------------
         (a+b+carry0) carry

(I find it interesting that this function accepts the carry from below the int args but returns it above the result.  Hmm...)

In [78]:
clear

[bool_to_int [0] [1] branch] inscribe

false bool_to_int
true bool_to_int

0 1

In [79]:
clear

false 1 2 [bool_to_int] dipd + +

3

In [80]:
clear

true 1 2 [bool_to_int] dipd + +

4

So the first part of `F` is `[bool_to_int] dipd + +` to get the total, then we need to

In [81]:
clear

4 base [mod] [>=] clop

4 false

In [82]:
clear

base 100 +

base [mod] [>=] clop

100 true

Put it all together and we have:

    _add-with-carry0 ≡ [bool_to_int] dipd + +
    _add-with-carry1 ≡ base [mod] [>=] clop

    add-with-carry ≡ _add-with-carry0 _add-with-carry1


In [83]:
clear
[_add-with-carry0 [bool_to_int] dipd + +] inscribe
[_add-with-carry1 base [mod] [>=] clop] inscribe
[add-with-carry _add-with-carry0 _add-with-carry1] inscribe

In [84]:
clear

false base 100 add-with-carry

100 true

In [85]:
clear

true base 100 add-with-carry

101 true

In [86]:
clear

false 2 100 add-with-carry

102 false

In [87]:
clear

true 2 100 add-with-carry

103 false

In [89]:
clear

### Now back to `add-carry-to-digits`

This should be a very simple recursive function.

    add-carry-to-digits ≡ [_actd_P] [_actd_THEN] [_actd_R0] [_actd_R1] genrec

    carry [n ...] add-carry-to-digits
    carry [n ...] [_actd_P] [_actd_THEN] [_actd_R0] [_actd_R1] genrec

The predicate is the carry flag itself inverted, but when we recur we need to check if the list is non-empty because it will eventually be empty or the carry flag will be false:

    _actd_P ≡ pop not

The base case simply discards the carry flag:

    _actd_THEN ≡ popd

That leaves the recursive branch:

    true [n ...] R0 [add-carry-to-digits] R1

-or-

    true [] R0 [add-carry-to-digits] R1

We know that the Boolean value is `true`.  We also know that the list will be non-empty, but only on the first iteration of the `genrec`.  It may be that the list is empty on a later iteration.

The `R0` function should check the list.

    _actd_R0 ≡ [bool] [_actd_R0.then] [_actd_R0.else] ifte

If it's empty...

       true [] R0.else [add-carry-to-digits] R1
    ----------------------------------------------
           false [1] [add-carry-to-digits] R1

    R0.else ≡ 1 swons popd false swap

If it's not empty...

       true [a ...] R0.then [add-carry-to-digits] R1
    -------------------------------------------------------------
       true 0 a add-with-carry [...1] [add-carry-to-digits] R1
    -------------------------------------------------------------
       (a+1) carry [...1] [add-carry-to-digits] R1

Ah ha!?  How to get that `(a+1)` back into the list!?

If it's empty...

       true [] R0.else [add-carry-to-digits] R1
    ------------------------------------------------
          1 false [] [add-carry-to-digits] i cons

and

       true [a ...] R0.then [add-carry-to-digits] i cons
    -------------------------------------------------------------
       true 0 a add-with-carry [...1] [add-carry-to-digits] i cons
    -------------------------------------------------------------
       (a+1) carry [...1] [add-carry-to-digits] i cons

There we go.

    _actd_R0.else ≡ popd 1 false rolldown

    _actd_R0.then ≡ 0 swap uncons [add-with-carry] dip

    _actd_R1 ≡ i cons

In [90]:
[add-carry-to-digits [_actd_P] [_actd_THEN] [_actd_R0] [_actd_R1] genrec] inscribe
[_actd_P pop not] inscribe
[_actd_THEN popd] inscribe
[_actd_R0 [bool] [_actd_R0.then] [_actd_R0.else] ifte] inscribe
[_actd_R0.else popd 1 false rolldown] inscribe
[_actd_R0.then 0 swap uncons [add-with-carry] dip] inscribe
[_actd_R1 i cons] inscribe

In [91]:
clear

false [3 2 1] add-carry-to-digits

[3 2 1]

In [92]:
clear

true [] add-carry-to-digits

[1]

In [93]:
clear

true [3 2 1] add-carry-to-digits

[4 2 1]

In [94]:
clear

true base -- [2 1] cons 

true [2147483647 2 1]

In [95]:
add-carry-to-digits

[0 3 1]

So that handles the cases where one of the two lists (but not both) is empty.

#### Both lists empty
If both lists are empty we discard one list and check the carry to determine our result as decribed above:

    ELSE ≡ pop swap [] [1 swons] branch

In [96]:
clear

true [] [] pop swap [] [1 swons] branch

[1]

In [97]:
clear

false [] [] pop swap [] [1 swons] branch

[]

In [98]:
clear

The story so far...

    add-digits ≡ initial-carry add-digits'
    
    add-digits' ≡ [P] [THEN] [R0] [R1] genrec

    initial-carry ≡ false rollup

    P ≡ [bool] ii & not

    THEN ≡ [P'] [THEN'] [ELSE] ifte

    P' ≡ [bool] ii |

    THEN' ≡ ditch-empty-list add-carry-to-digits

    ELSE ≡ pop swap [] [1 swons] branch

We just need to knock out those recursive functions `R0` and `R1` and we're done.

### And recur...

    bool [a ...] [b ...] R0 [add-digits'] R1

First we will want to `uncons` the digits

In [99]:
clear

false [1 2 3] [4 5 6] [uncons] ii swapd

false 1 4 [2 3] [5 6]

#### `uncons-two`
We could call this `uncons-two`:

In [100]:
clear

[uncons-two [uncons] ii swapd] inscribe

[1 2 3] [4 5 6] uncons-two

1 4 [2 3] [5 6]

In [102]:
clear

    bool a b [...] [...] R0' [add-digits'] R1

It's at this point that we'll want to employ the `add-with-carry` function:

    bool a b [...] [...] [add-with-carry] dipd R0'' [add-digits'] R1

    bool a b add-with-carry [...] [...] R0'' [add-digits'] R1

    (a+b) bool [...] [...] R0'' [add-digits'] R1

If we postulate a `cons` in our `R1` function...

    (a+b) bool [...] [...] R0'' [add-digits'] i cons

Then it seems like we're done?  `R0''` is nothing?

    R0 ≡ uncons-two [add-with-carry] dipd
    
    R1 ≡ i cons


In [103]:
[add-digits initial-carry add-digits'] inscribe
[add-digits' [P] [THEN] [R0] [R1] genrec] inscribe
[initial-carry false rollup] inscribe
[P [bool] ii & not] inscribe
[THEN [P'] [THEN'] [ELSE] ifte] inscribe
[P' [bool] ii |] inscribe
[THEN' ditch-empty-list add-carry-to-digits] inscribe
[ELSE pop swap [] [1 swons] branch] inscribe
[R0 uncons-two [add-with-carry] dipd] inscribe
[R1 i cons] inscribe


In [104]:
[3 2 1] [1 1 1]

[3 2 1] [1 1 1]

In [105]:
add-digits

[4 3 2]

In [106]:
base -- unit

[4 3 2] [2147483647]

In [107]:
add-digits

[3 4 2]

In [108]:
base -- dup dup unit ccons

[3 4 2] [2147483647 2147483647 2147483647]

In [109]:
add-digits

[2 4 2 1]

243 + 999 = 

In [110]:
clear 243 999 +

1242

Bitchin'

### Predicate
so the predicate for the whole thing would be

In [121]:
clear

[_add_p [bool not] ii &] inscribe

In [123]:
clear

[][]_add_p

true

In [124]:
clear

[1][]_add_p

false

In [125]:
clear

[][1]_add_p

false

In [126]:
clear

[1][1]_add_p

false

### Recursive Branch
So (ignoring the source of the initial carry bit for now) we have:

    _add_p == [bool not] ii &
    _add_then == pop swap [] [1 swons] branch

    add_digits == [_add_p] [_add_then] [R0] [R1] genrec

What should the recursive branch do?

It will be some `ifte` expression:

    R0 [add_digits] R1 == [P] [THEN] [ELSE] ifte

Ideally, we can arrange to neatly bracket the `[add_digits]` quote, but otherwise we can build up the `ifte` quotes in the usual way with `cons` and `concat`.

#### Is One List Empty?
The context will be a Boolean carry bit and two lists, one of which (but not both!) may be empty.

    carry [a ...] [b ...] R0 [add_digits] R1

    carry [] [b ...] R0 [add_digits] R1

    carry [a ...] [] R0 [add_digits] R1

So I think the first thing to do is detect and handle the (possibly) empty lists?

#### predicate

In [92]:
clear

[1] [1] [bool] ii &

true

In [93]:
clear

[1] [] [bool] ii &

false

In [94]:
clear

[] [1] [bool] ii &

false

So our `ifte` predicate is `[bool] ii &`, what's the rest look like?

    [[bool] ii &] ... [add_digits] ... ifte

We will need `add_digits` on the `true` branch to deal with the rest of the lists, but not (I think) on the false branch?

    [[bool] ii &] [... [add_digits] ...] [ELSE] ifte

In [130]:
[_add_rec_pred [bool] ii &] inscribe

Plugging in `add-with-carry` we get:

In [118]:
clear

false [1 2 3] [4 5 6] uncons-two [add-with-carry] dipd

5 false [2 3] [5 6]

### Recur

It is at this point that we would recur:

    5 false [2 3] [5 6] [add_digits] i ...
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 

This part runs and becomes some list (possibly empty) of result digits (the base case consumes the carry flag and one list):

    5 [7 9]

So our conclusion must be `cons`:

    5 [7 9] cons
    [5 7 9]

And our whole branch function is:

    THEN == uncons-two [add-with-carry] dipd [add_digits] i cons

    [_add_rec_pred] [THEN] [ELSE] ifte

It's a little tricky having that recursive call being nested like that:

    THEN0.1 == [add-with-carry] dipd
    THEN0 == uncons-two THEN0.1
    THEN1 == i cons

    [_add_rec_pred] [THEN0 [add_digits] THEN1] [ELSE] ifte

But we can make it work.

#### Interlude: Nested or Unnested?

    F == [P] [THEN] [R0]   [R1] genrec
    F == [P] [THEN] [R0 [F] R1] ifte

Our recursive branch will take the form of an `ifte` expression that incorporates the function `[F]` into one of its branches:

    F == [P] [THEN] [[P0] [THEN0   [F]   THEN1]             [ELSE] ifte] ifte

    F == [P] [THEN] [[P0] [THEN0]] [F] [[THEN1] cons concat [ELSE] ifte] ifte



       [P0] [THEN0] [F] [THEN1] cons concat [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0] [[F] THEN1]      concat [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0   [F] THEN1]            [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0 [F] THEN1] [ELSE] ifte

It's awkward but it works.

In [112]:
clear

#### add-carry-to-digits
So now we have `carry` and one list of digits.  (We also know that we have at least one digit in the list at this point but we likely won't need that information.)

    carry [a ...] add-carry-to-digits

I think once we define `add-carry-to-digits` we are pretty much done?  Let's find out.

##### If `carry` is `false` we just return the digits list, eh?

       false [a ...] swap [] [THEN] branch
    -----------------------------------------
             [a ...]

##### If `true` we want to maybe reuse `add-with-carry`?


       true [a ...] swap [THEN] [] branch
    ----------------------------------------
            [a ...]  THEN
    --------------------------------------
       false 1 a add-with-carry [...] G
    --------------------------------------
       (1+a) carry [...] G
    ------------------------------------------------
       (1+a) carry [...] add-carry-to-digits cons
    ------------------------------------------------
       (1+a)       [...]                     cons
    ------------------------------------------------
             [(1+a) ...]

So `add-carry-to-digits` is a recursive function (not a `branch` after all.)

Omitting `ditch-empty-list` for the moment:

    THEN == [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons

    [a ...] THEN
    [a ...] [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 [a ...]       uncons [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 a [...]              [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 a add-with-carry [...]                    [add-carry-to-digits] i cons
    (1+a) false              [...]                    [add-carry-to-digits] i cons
    (1+a) false              [...]                     add-carry-to-digits    cons
    (1+a)                    [...]                                            cons
    [(1+a) ...]

QED

In [64]:
clear

#### add-carry-to-digits, recursively

    carry [...] add-carry-to-digits

    carry [...] [P] [THEN] [R0] [R1] genrec

So:

    P == swap not
    
-or-

    P == pop not
    
and

    THEN == popd

That leaves:

    true [...] R0 [add-carry-to-digits] R1

I am tempted to leave the transformation from the above form to

    true [...] popd [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons

as the proverbial "exercise for the reader" but it's kinda trivial:

    R0 == popd [false 1] dip uncons [add-with-carry] dip
    R1 == i cons



In [65]:
true [1 2 3]  popd [false 1] dip uncons [add-with-carry] dip

2 false [2 3]

In [66]:
popd

2 [2 3]

In [67]:
cons

[2 2 3]

In [73]:
clear

[add-carry-to-digits [pop not] [popd] [popd [false 1] dip uncons [add-with-carry] dip] [i cons] genrec] inscribe

In [74]:
true [1 2 3] add-carry-to-digits

[2 2 3]

In [75]:
clear

false [1 2 3] add-carry-to-digits

[1 2 3]

In [84]:
clear

true base -- dup [2 3] ccons

true [2147483647 2147483647 2 3]

In [85]:
add-carry-to-digits

[0 0 3 3]

E.g. 3299 + 1 = 3300

In [90]:
clear

false base -- dup [2 3] ccons

false [2147483647 2147483647 2 3]

In [91]:
add-carry-to-digits

[2147483647 2147483647 2 3]

So that seems like it works.

In [119]:
clear

## Our Story So Far...
    _add_p == [bool not] ii &
    _add_then == pop swap [] [1 swons] branch
    _add_rec_pred == [bool] ii &

    add_digits == [_add_p] [_add_then] [R0] [R1] genrec

Where:

                    R0 [add_digits] R1
    -----------------------------------------------------
       [_add_rec_pred] [          THEN          ] [ELSE] ifte
    -----------------------------------------------------
       [_add_rec_pred] [THEN0 [add_digits] THEN1] [ELSE] ifte

    THEN0 == uncons-two [add-with-carry] dipd

    THEN1 == i cons

    ELSE == ditch-empty-list add-carry-to-digits

In [131]:
[THEN0 uncons-two [add-with-carry] dipd] inscribe
[THEN1 i cons] inscribe
[ELSE ditch-empty-list add-carry-to-digits] inscribe

Recalling:

       [P0] [THEN0] [F] [THEN1] cons concat [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0] [[F] THEN1]      concat [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0   [F] THEN1]            [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0 [F] THEN1] [ELSE] ifte

Ergo:

    R0 == [_add_rec_pred] [THEN0]
    R1 == [THEN1] cons concat [ELSE] ifte

    add_digits == [_add_p] [_add_then] [R0] [R1] genrec


In [132]:
[_add_R0 [_add_rec_pred] [THEN0]] inscribe
[_add_R1 [THEN1] cons concat [ELSE] ifte] inscribe

In [133]:
[add_digits [_add_p] [_add_then] [_add_R0] [_add_R1] genrec] inscribe

In [134]:
false [3 2 1] [8] 

false [3 2 1] [8]

In [135]:
add_digits

[11 2 1]

In [136]:
base -- swons

[2147483647 11 2 1]

In [137]:
false swap [23]

false [2147483647 11 2 1] [23]

In [138]:
add_digits

[22 12 2 1]

In [142]:
clear base -- unit [1]

[2147483647] [1]

When i tried it it failed and inspection revealed that I had forgotten to set the initial carry value.

In [144]:
clear false base -- unit [1]

false [2147483647] [1]

In [145]:
add_digits

[0 1]

In [148]:
clear false base -- unit dup concat dup concat [1]

false [2147483647 2147483647 2147483647 2147483647] [1]

#### I made a mistake in add-carry-to-digits

The recursive branch:

    true [...] R0 [add-carry-to-digits] R1

Must also deal with the case where the list is empty!

    true [] R0 [add-carry-to-digits] R1

The `popd` is okay, but then we need a branch:

    R0 == popd ? [1 swons] [R0.t] branch
    R0.t == [false 1] dip uncons [add-with-carry] dip

    R1 == i cons



#### add-carry-to-digits, recursively, TAKE TWO

    carry [...] add-carry-to-digits

    carry [...] [P] [THEN] [R0] [R1] genrec

    P == pop not
    THEN == popd

That leaves one of:

    true [...] R0 [add-carry-to-digits] R1

-or-

    true [] R0 [add-carry-to-digits] R1

It would be fun to just set `R1` to `ifte`

    true [] R0 [add-carry-to-digits] ifte

Which means that `R0` must be a predicate and the "then" branch.

    true [] [bool not] [THEN] [add-carry-to-digits] ifte

And I think `THEN` would just be:

    THEN == popd 1 swons




In [162]:
clear
[R0.t [false 1] dip uncons [add-with-carry] dip] inscribe
[add-carry-to-digits
  [pop not]
  [popd]
  [[bool not] [popd 1 swons]]
  [ifte]
  genrec
] inscribe

This is still broken because the 

In [160]:
clear true base -- unit dup concat dup concat

true [2147483647 2147483647 2147483647 2147483647]

In [ ]:
add-carry-to-digits

#### add-carry-to-digits

So `add-carry-to-digits` is a recursive function:

    carry [a ...] add-carry-to-digits

    carry [a ...] [P] [THEN] [R0] [R1] genrec



In [ ]:
[pop not] [popd] [[bool not] [1 cons swap]] [ifte] genrec

##### If `carry` is `false` we just return the digits list, eh?

       false [a ...] [pop not] [popd] [R0] [R1] genrec
    -----------------------------------------------------
             [a ...]

##### If `true` we want to maybe reuse `add-with-carry`?

    true [a ...] R0 [add-carry-to-digits] R1

-or-

    true [] R0 [add-carry-to-digits] R1

Um...

    true [] [bool not] [1 cons popd] [add-carry-to-digits] ifte


    true [] [bool not] [1 cons popd] [add-carry-to-digits] ifte
    true [] 1 cons popd
    true [1] popd
    [1]

-or-

    true [a ...] [bool not] [1 cons swap] [F add-carry-to-digits] itfe
    true [a ...] F add-carry-to-digits


       true                  [a ...] F add-carry-to-digits
    ---------------------------------------------------------
       true 0 a add-with-carry [...]   add-carry-to-digits

    
    F == 0 swap uncons [add-with-carry] dip


    true [a ...] [bool not] [1 cons swap] [add-carry-to-digits] [F] swoncat itfe



    true [a ...] [bool not] [1 cons swap] [add-carry-to-digits] [0 swap uncons [add-with-carry] dip] swoncat itfe



In [162]:
clear
[R0.t [false 1] dip uncons [add-with-carry] dip] inscribe
[add-carry-to-digits
  [pop not]
  [popd]
  [[bool not] [popd 1 swons]]
  [ifte]
  genrec
] inscribe

Omitting `ditch-empty-list` for the moment:

    THEN == [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons

    [a ...] THEN
    [a ...] [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 [a ...]       uncons [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 a [...]              [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 a add-with-carry [...]                    [add-carry-to-digits] i cons
    (1+a) false              [...]                    [add-carry-to-digits] i cons
    (1+a) false              [...]                     add-carry-to-digits    cons
    (1+a)                    [...]                                            cons
    [(1+a) ...]

QED

In [64]:
clear

#### add-carry-to-digits, recursively

    carry [...] add-carry-to-digits

    carry [...] [P] [THEN] [R0] [R1] genrec

So:

    P == swap not
    
-or-

    P == pop not
    
and

    THEN == popd

That leaves:

    true [...] R0 [add-carry-to-digits] R1

I am tempted to leave the transformation from the above form to

    true [...] popd [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons

as the proverbial "exercise for the reader" but it's kinda trivial:

    R0 == popd [false 1] dip uncons [add-with-carry] dip
    R1 == i cons



In [ ]:
[base 2147483648] inscribe
[valid_digit [0 >] [base <] &&] inscribe
[_add_p [bool not] ii &] inscribe
[_add_then pop swap [] [1 swons] branch] inscribe
[_add_rec_pred [bool] ii &] inscribe
[bool_to_int [0] [1] branch] inscribe
[_add-with-carry0 [bool_to_int] dipd + +] inscribe
[_add-with-carry1 base [mod] [>=] clop] inscribe
[add-with-carry _add-with-carry0 _add-with-carry1] inscribe
[uncons-two [uncons] ii swapd] inscribe
[ditch-empty-list [bool] [popd] [pop] ifte] inscribe
[add-carry-to-digits [pop not] [popd] [popd [false 1] dip uncons [add-with-carry] dip] [i cons] genrec] inscribe
[THEN0 uncons-two [add-with-carry] dipd] inscribe
[THEN1 i cons] inscribe
[ELSE ditch-empty-list add-carry-to-digits] inscribe
[_add_R0 [_add_rec_pred] [THEN0]] inscribe
[_add_R1 [THEN1] cons concat [ELSE] ifte] inscribe
[add_digits [_add_p] [_add_then] [_add_R0] [_add_R1] genrec] inscribe


In [55]:
[isnt_bigint
[first isnt_bool]
[rest [isnt_int] map]
cleave cons
] inscribe

[1] 2 false [2 3]

In [56]:
words

[1] 2 false [2 3] [[!] [!-] [!=] [%] [&] [&&] [*] [+] [++] [-] [--] [/] [<] [<<] [<<{}] [<=] [<>] [<{}] [=] [>] [>=] [>>] [?] [abs] [add] [add-with-carry] [all] [anamorphism] [and] [any] [app1] [app2] [app3] [appN] [at] [b] [base] [binary] [bool] [bool_to_int] [branch] [ccccons] [ccons] [choice] [clear] [cleave] [clop] [cmp] [codi] [codireco] [concat] [cond] [cons] [digitalize] [dinfrirst] [dip] [dipd] [dipdd] [disenstacken] [ditch-empty-list] [div] [divmod] [down_to_zero] [drop] [dup] [dupd] [dupdd] [dupdip] [dupdipd] [enstacken] [eq] [first] [first_two] [flatten] [floordiv] [fork] [fourth] [gcd] [gcd2] [ge] [genrec] [getitem] [grabN] [grba] [gt] [help] [i] [ifte] [ii] [infra] [infrst] [inscribe] [isnt_bigint] [isnt_bool] [isnt_int] [isnt_stack] [le] [loop] [lshift] [lt] [make_generator] [manual] [map] [max] [min] [mod] [mul] [ne] [neg] [not] [nulco] [nullary] [of] [or] [over] [pam] [pm] [pop] [popd] [popdd] [popop] [popopd] [popopdd] [popopop] [pow] [pred] [primrec] [product] [quit] 

In [57]:
clear

In [58]:
1 31 <<

2147483648

In [59]:
clear

123 10 divmod

12 3

In [60]:
base divmod

12 0 3

In [61]:
[divmod] help


==== Help on divmod ====

Similarly to pm ("Plus or minus") this function computes
both the
::

          a b divmod
    ---------------------
       a b div a b mod
    ---------------------
             q r

Where: q * b + r == a

---- end ( divmod )


12 0 3